# Домашнее задание 4. Конструирование текстовых признаков из твитов пользователей

## Сбор данных

Первый этап - сбор твитов пользователей. Необходимо подключаться к Twitter API и запрашивать твиты по id пользователя. 
Подключение к API подробно описано в ДЗ 1.

In [10]:
import twitter

CONSUMER_KEY = "RAhs7ThpKFL14Nb188tdkdLkX"
CONSUMER_SECRET = "V7VaBOgAnOnEk0Q3v9A0MKygAYwJqboNtFfsURIP52Cp2ar2EX"

ACCESS_TOKEN_KEY = "430060558-rBilaliqRUoDm9JHTeSsWUpM3BsCoDvtKhtul5On"
ACCESS_TOKEN_SECRET = "iqHidUrd9tstB97CWx6fw1xg7gbeAd4QibrFcL9d5JflA"

api = twitter.Api(consumer_key=CONSUMER_KEY,
                  consumer_secret=CONSUMER_SECRET,
                  access_token_key=ACCESS_TOKEN_KEY,
                  access_token_secret=ACCESS_TOKEN_SECRET,
                  sleep_on_rate_limit=True)

Для получения твитов пользователя может быть использован метод GetUserTimeline из библиотеки python-twitter. Он позволяет получить не более 200 твитов пользователя. По каждому пользователю достаточно собрать 200 твитов.

Метод имеет ограничение по количеству запросов в секунду. Для получения информации о промежутке времени, которое необходимо подождать для повторного обращения к API может быть использован метод `GetSleepTime`. Для получения информации об ограничениях запросов с помощью метода `GetUserTimeLine` необходимо вызывать `GetSleepTime` с параметром "statuses/user_timeline".

Метод GetUserTimeline возвращает объекты типа Status. У этих объектов есть метод AsDict, который позволяет представить твит в виде словаря.

Id пользователей необходимо считать из файла, как было сделано в ДЗ 1.

Необходимо реализовать функцию `get_user_tweets(user_id)`. Входной параметр - id пользователя из файла. Возвращаемое значение - массив твитов пользователя, где каждый твит представлен в виде словаря. Предполагается, что информация о пользователе содержится в твитах, которые пользователь написал сам. Это означает, что можно попробовать отфильтровать ответы другим пользователям, ссылки и ретвиты, а так же картинки и видео, так как наша цель - найти текстовую информацию.

In [11]:
import string, re

MAX_NUMBER_PER_REQUEST = 200

def get_user_tweets(user_id):
    records = []
    try:
        user_tweets = api.GetUserTimeline(user_id=user_id, count=MAX_NUMBER_PER_REQUEST,
                                          exclude_replies=True, include_rts=False)
        for tweet in user_tweets:
            tweet = tweet.AsDict()
            if tweet[u'lang'] == u'en' and not u'media' in tweet:
                urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', tweet["text"])
                mentions = re.findall('@(?:[a-zA-Z]|[0-9]|[_])+', tweet["text"])    
                for url_or_mention in urls + mentions:
                    tweet["text"] = tweet["text"].replace(url_or_mention, "")
                records.append({u'text': tweet[u'text']})
    except:
        print "Exception: could not get tweets for user:{}".format(user_id)
    return records

## Разбор текста твита

Обработка текста предполагает разбиение текста на отдельные элементы - параграфы, предложения, слова. Мы будем преобразовывать текст твита к словам. Для этого текст необходимо разбить на слова. Сделать это можно, например, с помощью регулярного выражения.

Необходимо реализовать функцию, `get_words(text)`. Входной параметр - строка с текстом. Возвращаемое значение - массив строк (слов). Обратите внимание, что нужно учесть возможное наличие пунктуации и выделить по возможности только слова. 

In [12]:
def get_words(text):
    """returns list of words"""
    text_list = re.findall(r"[\w']+|[.,!?;]", text)
    punctuation = set(string.punctuation)
    return [w for w in text_list if w not in punctuation]

Далее полученные слова необходимо привести к нормальной форме. То есть привести их к форме единственного числа настоящего времени и пр. Сделать это можно с помощью библиотеки nltk. Информацию по загрузке, установке библиотеки и примерах использования можно найти на сайте http://www.nltk.org/

Для загрузки всех необходимых словарей можно воспользоваться методом download из библиотеки nltk.

In [13]:
import nltk
#nltk.download()

Для дальнейшей обработки слова должны быть приведены к нижнему регистру. 

Для приведения к нормальной форме можно использовать `WordNetLemmatizer` из библиотеки nltk. У этого класса есть метод `lemmatize`.

Также необходимо убрать из текста так называемые стоп-слова. Это часто используемые слова, не несущие смысловой нагрузки для наших задач. Сделать это можно с помощью `stopwords` из nltk.corpus

Необходимо реализовать функцию `get_tokens(words)`. Входной параметр - массив слов. Возвращаемое значение - массив токенов.

In [14]:
import unicodedata
from nltk.corpus import stopwords

def get_tokens(words):
    """returns list of tokens"""
    tokens = []
    stop = set(stopwords.words('english'))
    for w in words:
        normalized_w = ''.join(c for c in unicodedata.normalize('NFD', w.lower ()) 
                                if not unicodedata.combining (c))
        if normalized_w not in stop:
            tokens.append(normalized_w)
    return tokens

Необходимо реализовать функцию `get_tweet_tokens(tweet)`. Входной параметр - текст твита. Возвращаемое значение -- токены твита. 

In [15]:
def get_tweet_tokens(tweet):
    return get_tokens(tweet)

Необходимо реализовать функцию `collect_users_tokens()`. Функция должна сконструировать матрицу признаков пользователей. В этой матрице строка - пользователь. Столбец - токен. На пересечении - сколько раз токен встречается у пользователя.
Для построения матрицы можно использовать `DictVectorizer` из `sklearn.feature_extraction`.

In [16]:
from collections import Counter

def collect_users_tokens(df_users):
    """returns users list and list of user dicts. Each dict contains frequence of user tokens"""
    counter = 0
    users = []
    users_tokens = []
    for u in df_users[u'uid']:
        tw_tokens = []
        tweets = get_user_tweets(u)    
        for tw in tweets:
            tw_words = get_words(tw[u'text'])
            tw_tokens.extend(get_tweet_tokens(tw_words))
                    
        record = Counter(tw_tokens)
        record = dict(record)

        if record:
            users_tokens.append(dict(record))
            users.append(u)
        
        counter += 1
        if counter < 32 and counter % 2 == 0:
            print counter
        elif counter < 64 and counter % 4 == 0:
            print counter
        elif counter < 128 and counter % 8 == 0:
            print counter
        elif counter < 256 and counter % 16 == 0:
            print counter
        elif counter < 512 and counter % 32 == 0:
            print counter
        elif counter < 1024 and counter % 64 == 0:
            print counter
        elif counter < 2048 and counter % 128 == 0:
            print counter
        elif counter < 4096 and counter % 256 == 0:
            print counter
        elif counter < 8192 and counter % 512 == 0:
            print counter
    return users, users_tokens

In [17]:
import json
import pandas as pd
from collections import defaultdict
from sklearn.feature_extraction import DictVectorizer


TRAINING_SET_URL = "twitter_train.csv"
EXAMPLE_SET_URL = "twitter_example.csv"
df_users_train = pd.read_csv(TRAINING_SET_URL, sep=",", header=0, names=["uid", "cat"])
df_users_ex = pd.read_csv(EXAMPLE_SET_URL, sep=",", header=0, names=["uid"])
df_users_ex['cat'] = None
df_users = pd.concat([df_users_train, df_users_ex])

users, users_tokens = collect_users_tokens(df_users)
v = DictVectorizer()
vs = v.fit_transform(users_tokens)

2
4
6
8
10
12
14
16
18
20
Exception: could not get tweets for user:2330149164
22
24
26
28
30
32
36
40
44
48
52
56
60
64
72
80
88
96
104
112
120
128
Exception: could not get tweets for user:3091562110
144
Exception: could not get tweets for user:3091636366
160
176
192
208
224
240
256
Exception: could not get tweets for user:760201676187918336
288
320
352
384
Exception: could not get tweets for user:4249177700
416
448
Exception: could not get tweets for user:904072386
Exception: could not get tweets for user:119119142
Exception: could not get tweets for user:121606839
Exception: could not get tweets for user:1622658864
480
Exception: could not get tweets for user:969130340
512
Exception: could not get tweets for user:756514146
Exception: could not get tweets for user:1027382700
576
Exception: could not get tweets for user:712177938259640320
640
Exception: could not get tweets for user:143337751
704
Exception: could not get tweets for user:228203145
Exception: could not get tweets for use

In [18]:
import numpy as np
np.savez("out_4.dat", data=vs, users=users, users_tokens=users_tokens )

Далее для получения представления о полученной информацию о токенах предлагается отобразить ее в виде облака тэгов. [Подсказка](http://anokhin.github.io/img/tag_cloud.png). 

In [20]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

def draw_tag_cloud(v, vs):
    text = ""
    user_tokens = v.inverse_transform(vs)
    words = Counter()

    for tokens in user_tokens:
        words += Counter(tokens)
        
    most_common = words.most_common(100)
    
    for common_tuple in most_common:
        text += (common_tuple[0] + " ") * int(common_tuple[1])
        
    wordcloud = WordCloud(background_color='white').generate(text)
    
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.savefig('wordcloud.png', dpi=500)
    plt.show()
        
    return

In [21]:
draw_tag_cloud(v, vs)